In [1]:
import torch
import transformers
from ast import literal_eval
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import Dataset
import json
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import evaluate
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from peft import AutoPeftModelForCausalLM, LoraConfig
from rag import retrieve_query,init_vectorstore
from pprint import pprint

pd.set_option('display.max_columns', None)

In [2]:
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42) # magic number :)

In [3]:
PROMPT_NO_QUESTION_PLUS_4 = """지문:
{paragraph}

질문:
{question}

선택지:
{choices}

먼저 문제를 이해하고, 문제 해결을 위하여 계획을 세워보세요.
그 다음 단계별로 생각하여 정답을 고르세요.
1, 2, 3, 4 중에 하나를 정답으로 고르세요.
정답:"""

PROMPT_QUESTION_PLUS_4 = """지문:
{paragraph}

질문:
{question}

<보기>:
{question_plus}

선택지:
{choices}

먼저 문제를 이해하고, 문제 해결을 위하여 계획을 세워보세요.
그 다음 단계별로 생각하여 정답을 고르세요.
1, 2, 3, 4 중에 하나를 정답으로 고르세요.
정답:"""

PROMPT_NO_QUESTION_PLUS_5 = """지문:
{paragraph}

질문:
{question}

선택지:
{choices}

먼저 문제를 이해하고, 문제 해결을 위하여 계획을 세워보세요.
그 다음 단계별로 생각하여 정답을 고르세요.
1, 2, 3, 4, 5 중에 하나를 정답으로 고르세요.
정답:"""

PROMPT_QUESTION_PLUS_5 = """지문:
{paragraph}

질문:
{question}

<보기>:
{question_plus}

선택지:
{choices}
먼저 문제를 이해하고, 문제 해결을 위하여 계획을 세워보세요.
그 다음 단계별로 생각하여 정답을 고르세요.
1, 2, 3, 4, 5 중에 하나를 정답으로 고르세요.
정답:"""

In [3]:
PROMPT_NO_QUESTION_PLUS = """<문제1>:
지문: 현대 사회에서 플라스틱 사용량이 급증하면서 환경 오염이 심각한 문제로 대두되고 있다. 특히 해양 생태계에서 미세플라스틱으로 인한 피해가 갈수록 증가하고 있다...
질문: 위 글에서 설명하는 환경 문제의 주된 원인은?
선택지:
1 - 화석연료의 과다 사용
2 - 플라스틱 사용량 증가
3 - 수질 오염
4 - 대기 오염
[분석]
지문 요약
현대 사회의 플라스틱 사용 증가와 이로 인한 환경 오염, 특히 해양 생태계 피해를 다루고 있음

질문 분석

환경 문제의 주된 원인을 찾아야 함
지문에서 직접적으로 언급된 원인을 찾아야 함

선택지 분석
1 - 화석연료 - 지문에서 언급되지 않음
2 - 플라스틱 사용량 증가 - 지문에서 직접적으로 언급됨
3 - 수질 오염 - 결과적 현상이며 원인이 아님
4 - 대기 오염 - 지문에서 언급되지 않음

정답:2번 
근거: "현대 사회에서 플라스틱 사용량이 급증하면서 환경 오염이 심각한 문제로 대두되고 있다"라는 문장에서 직접적으로 언급됨

<문제2>:
지문:
{paragraph}

질문:
{question}

선택지:
{choices}


정답:"""

PROMPT_QUESTION_PLUS = """<문제1>:
지문: 현대 사회에서 플라스틱 사용량이 급증하면서 환경 오염이 심각한 문제로 대두되고 있다. 특히 해양 생태계에서 미세플라스틱으로 인한 피해가 갈수록 증가하고 있다...
질문: 위 글에서 설명하는 환경 문제의 주된 원인은?
선택지:
1 - 화석연료의 과다 사용
2 - 플라스틱 사용량 증가
3 - 수질 오염
4 - 대기 오염
[분석]
지문 요약
현대 사회의 플라스틱 사용 증가와 이로 인한 환경 오염, 특히 해양 생태계 피해를 다루고 있음

질문 분석

환경 문제의 주된 원인을 찾아야 함
지문에서 직접적으로 언급된 원인을 찾아야 함

선택지 분석
1 - 화석연료 - 지문에서 언급되지 않음
2 - 플라스틱 사용량 증가 - 지문에서 직접적으로 언급됨
3 - 수질 오염 - 결과적 현상이며 원인이 아님
4 - 대기 오염 - 지문에서 언급되지 않음

정답:2번 
근거: "현대 사회에서 플라스틱 사용량이 급증하면서 환경 오염이 심각한 문제로 대두되고 있다"라는 문장에서 직접적으로 언급됨

<문제2>:
지문:
{paragraph}

질문:
{question}

<보기>:
{question_plus}

선택지:
{choices}


정답:"""

In [4]:
# Load the test dataset
test_df = pd.read_csv('../data/test.csv')

# Flatten the JSON dataset
records = []
for _, row in test_df.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
test_df = pd.DataFrame(records)

In [6]:
test_dataset = []
for i, row in test_df.iterrows():
    choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(row["choices"])])
    len_choices = len(row["choices"])
    
    # <보기>가 있을 때
    if row["question_plus"]:
        if len_choices == 4:
            user_message = PROMPT_QUESTION_PLUS.format(
                paragraph=row["paragraph"],
                question=row["question"],
                question_plus=row["question_plus"],
                choices=choices_string,
            )
        elif len_choices == 5:
            user_message = PROMPT_QUESTION_PLUS.format(
                paragraph=row["paragraph"],
                question=row["question"],
                question_plus=row["question_plus"],
                choices=choices_string,
            )
    # <보기>가 없을 때
    else:
        if len_choices == 4:
            user_message = PROMPT_NO_QUESTION_PLUS.format(
                paragraph=row["paragraph"],
                question=row["question"],
                choices=choices_string,
            )
        elif len_choices == 5:
            user_message = PROMPT_NO_QUESTION_PLUS.format(
                paragraph=row["paragraph"],
                question=row["question"],
                choices=choices_string,
            )

    test_dataset.append(
        {
            "id": row["id"],
            "messages": [
                {"role": "system", "content": "지문을 보고 정답을 찾을 수 있는지 판별하세요."},
                {"role": "user", "content": user_message},
            ],
            "label": row["answer"],
            "len_choices": len_choices,
        }
    )

In [7]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4")

sampling_params = SamplingParams(temperature=1, top_p=0.8, repetition_penalty=1.05, max_tokens=1024)
params = {
    "max_model_len": 4096,
}
llm = LLM(model="Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4" , **params)


WARNING 11-28 00:47:48 config.py:321] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 11-28 00:47:48 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4', speculative_config=None, tokenizer='Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_t

/data/ephemeral/home/.venv/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/data/ephemeral/home/.venv/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


INFO 11-28 00:47:50 model_runner.py:1056] Starting to load model Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4...
INFO 11-28 00:47:51 selector.py:224] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 11-28 00:47:51 selector.py:115] Using XFormers backend.
INFO 11-28 00:47:52 weight_utils.py:243] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


INFO 11-28 00:47:59 model_runner.py:1067] Loading model weights took 18.1434 GB
INFO 11-28 00:48:03 gpu_executor.py:122] # GPU blocks: 1975, # CPU blocks: 1024
INFO 11-28 00:48:03 gpu_executor.py:126] Maximum concurrency for 4096 tokens per request: 7.71x
INFO 11-28 00:48:06 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-28 00:48:06 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-28 00:48:49 model_runner.py:1523] Graph capturing finished in 43 secs.


In [8]:
answers = []

for i in tqdm(range(len(test_dataset))):
    prompt = test_dataset[i]['messages'][1]['content']
    messages = [
        {"role": "system", "content": "당신은 문제를 매우 잘푸는 선생입니다.문제를 읽고 정답을 고르세요."},
        {"role": "user", "content": prompt}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        add_generation_prompt=True
    )
    outputs = llm.generate(text, sampling_params)
    for output in outputs:
        answers.append(output.outputs[0].text)

100%|██████████| 869/869 [2:10:37<00:00,  9.02s/it]


In [9]:
# Save the answers
answers_df = pd.DataFrame(columns=["id", "answer"])
answers_df["id"] = test_df["id"]
answers_df["answer"] = answers


In [10]:
answers_df.to_csv("qwen_vllm_answers_1shot.csv", index=False)